In [6]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
from torch.nn import CrossEntropyLoss
import numpy as np
import json
import pickle
from transformers import StoppingCriteria, StoppingCriteriaList
import re
from vicuna_setup import vicuna_inference
from vicuna_pubmedqa_eval import model_pubmed_evaluator
import pandas as pd
import csv
from collections import Counter
from evaluation import pub_scores


In [7]:
#torch.cuda.empty_cache()

In [8]:
model_path="/home/vicuna-weights-7B"
peft_path="/home/medical-llama/vicuna-lora-7B"
tokenizer = LlamaTokenizer.from_pretrained(model_path)

In [9]:
with open("/home/medical-llama/vicuna-lora-7B/adapter_config.json", 'r') as json_file:
    config = json.load(json_file)

In [10]:
peft_config = PeftConfig.from_pretrained(peft_path)


In [11]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = LlamaForCausalLM.from_pretrained(model_path)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.81s/it]


In [12]:
model = PeftModel.from_pretrained(
        model,
        peft_path,
        torch_dtype=torch.float16,
)

In [13]:
# PUBMEDQA EVALUATION

In [14]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2


In [15]:


pubmed_config=config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=3,
        min_length=1,
        max_time=3,
        max_new_tokens=3, # 3
        forced_bos_token_id=[tokenizer.encode("yes")[1], tokenizer.encode("no")[1], tokenizer.encode("maybe")[1]],
        
    )

In [16]:
ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=0)

In [17]:
len(ans_dict)

500

In [18]:
with open("/home/medical-llama/pubmed-pred/fine-tuned.json", "w") as json_file:
    json.dump(ans_dict, json_file)

In [19]:
pub_scores("/home/medical-llama/pubmed-pred/fine-tuned.json")


(4.3999999999999995, 3.364222540977989)

In [ ]:
acc,f1_macro=[],[]
for i in range(5):
    ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=i)
    with open("/home/medical-llama/pubmed-pred/fine-tuned.json", "w") as json_file:
        json.dump(ans_dict, json_file)
    accuracy,f1=pub_scores("/home/medical-llama/pubmed-pred/fine-tuned.json")
    acc.append(accuracy)
    f1_macro.append(f1)

In [ ]:
print(acc)
print(f1_macro)

In [ ]:
print("Accuracy: ",np.mean(acc))
print("F1 Macro: ",np.mean(f1_macro))

In [34]:
vicuna_inference("what is gradient descent ?",model=model,tokenizer=tokenizer,config=pubmed_config)

'Gradient descent is an optimization algorithm used to minimize a cost\nfunction by iteratively adjusting the parameters of a model in the\ndirection of steepest descent, which is the negative of the gradient\nof the cost function with respect to the parameters. The gradient\ndescent algorithm starts with an initial set'

In [41]:
ans_dict

{'12377809': 'no',
 '26163474': '',
 '19100463': '',
 '18537964': '',
 '12913878': '1.',
 '12765819': '',
 '25475395': '',
 '19130332': 'no',
 '9427037': 'no',
 '24481006': '',
 '8165771': '',
 '22680064': '1.',
 '22540518': '',
 '20629769': 'yes',
 '21726930': '',
 '21481154': 'yes',
 '22902073': '',
 '26370095': '',
 '18041059': '',
 '15041506': 'no',
 '11146778': '',
 '27281318': '',
 '21645374': '',
 '9465206': '',
 '25887165': '',
 '15995461': '',
 '21850494': '',
 '19106867': '1.',
 '21342862': '',
 '24352924': '',
 '16147837': '',
 '26879871': '',
 '15918864': '',
 '22075911': 'no',
 '11035130': '',
 '21228436': '',
 '11833948': '',
 '17682349': 'no',
 '17355582': 'no',
 '15597845': '1.',
 '10158597': 'no',
 '27549226': '',
 '26348845': '',
 '25588461': '1.',
 '23359100': '',
 '26548832': '',
 '25756710': '',
 '20297950': '',
 '24622801': '',
 '9722752': '',
 '20577124': '',
 '19027440': '',
 '18239988': '',
 '27858166': '',
 '27050489': 'no',
 '16266387': '',
 '27287237': 'yes'

In [ ]:
vicuna_inference("what is gradient descent ?",model=model,tokenizer=tokenizer,config=pubmed_config)

In [14]:
def pubmed_template(Question, Context,id=0):
    if id==0:
        prompt_template=f"""

    Respond with ""yes , no , maybe "" using the context & the question provided.    

    ###
            
    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are dogs trainable?

    Answer: yes

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?

    Answer: maybe

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are Dogs highly introverted & reserved ?

    Answer: no 

    ###

    Context:
    {Context}

    Question:
    {Question}

    Answer: 
    
    """
        return prompt_template

In [ ]:
def pubmed_template(Question, Context,id=0):

    prompt_template=f"""
    
    your task is to answering research questions relating to medical domain using yes/no/maybe responses. Answer as yes if the context supports the question , answer as no if the context does not support the question , answer as maybe if the context is not clear enough to answer the question.      

    ###
            
    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are dogs trainable?

    Answer: yes

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?

    Answer: maybe

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are Dogs highly introverted & reserved ?

    Answer: no 

    ###

    Context:
    {Context}

    Question:
    {Question}

    Answer: 
    """
    return prompt_template

In [16]:
df = pd.read_csv("/home/medical-llama/Pubmedqa/pubmedqa/data/pubmedqa_test.csv")


In [28]:
ans_dict={}
for i in range(int(df.shape[0]/100)):
    pmid=str(df['ID'][i])
    ans=vicuna_inference(pubmed_template(Question=df['Question'][i],Context=df['Context'][i]),model=model,tokenizer=tokenizer,config=pubmed_config)
    print(ans)

"no



1.


In [25]:

ans_dict={}
for i in range(int(df.shape[0]/10)):
    pmid=str(df['ID'][i])
    ans=vicuna_inference(pubmed_template(Question=df['Question'][i],Context=df['Context'][i]),model=model,tokenizer=tokenizer,config=pubmed_config)
    #print(ans)
    ans_dict[pmid]=ans

    ans_dict=output_preprocessing(ans_dict)


In [26]:
ans_dict

{'12377809': 'no',
 '26163474': 'yes',
 '19100463': '',
 '18537964': 'yes',
 '12913878': '1.',
 '12765819': '',
 '25475395': 'yes',
 '19130332': 'no',
 '9427037': 'no',
 '24481006': '',
 '8165771': '-',
 '22680064': '1.',
 '22540518': 'yes',
 '20629769': 'yes',
 '21726930': 'yes',
 '21481154': 'yes',
 '22902073': '',
 '26370095': 'yes',
 '18041059': '',
 '15041506': 'no',
 '11146778': '',
 '27281318': '',
 '21645374': '',
 '9465206': '',
 '25887165': '',
 '15995461': '',
 '21850494': 'yes',
 '19106867': '1.',
 '21342862': 'yes',
 '24352924': '',
 '16147837': '',
 '26879871': '',
 '15918864': 'yes',
 '22075911': 'no',
 '11035130': 'yes',
 '21228436': '',
 '11833948': '',
 '17682349': 'no',
 '17355582': 'no',
 '15597845': '1.',
 '10158597': 'no',
 '27549226': 'yes',
 '26348845': '',
 '25588461': '1.',
 '23359100': '',
 '26548832': '',
 '25756710': 'yes',
 '20297950': '',
 '24622801': 'yes',
 '9722752': 'yes'}